# Gaussian Naive Bayes Classifier

In [17]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import confusion_matrix

## GaussianNB Classifier
An implementation of a Gaussian Naive Bayes that fits the framework.

In [18]:
class MyGaussianNB(BaseEstimator, ClassifierMixin):
    def fit(self, xt, yt):
        xt = np.array(xt)
        yt = np.array(yt)

        # first phase: compute class priors
        self._classes, self._class_priors = np.unique(yt, return_counts=True)
        self._class_priors = self._class_priors / float(np.sum(self._class_priors))  

        # second phase: compute all features' means and variances
        xt_split = [xt[yt==cl] for cl in self._classes]
        self._means = np.array([np.mean(xtcl, axis=0) for xtcl in xt_split])
        self._vars = np.array([np.var(xtcl, axis=0) for xtcl in xt_split])
        return self

    def predict(self, xtests):
        xtests = np.array(xtests)

        probas = self._compute_probas(xtests)
        return np.array([self._classes[i] for i in np.argmax(probas, axis=0)])
        
    def predict_proba(self, xtests):
        xtests = np.array(xtests)

        probas = self._compute_probas(xtests)
        return probas / np.sum(probas, axis=0)

    def _compute_probas(self, xtests):
        return np.array([
            [
                self._class_priors[i] * np.product(self._conditional_proba(xtest, self._means[i], self._vars[i])) 
                for xtest in xtests
            ]
            for i in range(len(self._classes))
        ])
    
    @staticmethod
    def _conditional_proba(xt, m, var):
        return np.exp(-np.power(xt-m, 2)/(2*var))/np.sqrt(2*np.pi*var)

## Penguins dataset
Running Gaussian Naive Bayes on the penguin dataset.

In [19]:
penguins_af = pd.read_csv('penguins_af.csv', index_col = 0)
print(penguins_af.shape)
penguins_af.head()

(333, 8)


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007


In [24]:
f_names = ['bill_length_mm', 'bill_depth_mm','flipper_length_mm', 'body_mass_g']
penguins = penguins_af[f_names + ['species']]
# penguins2C = penguins.loc[penguins['species'].isin(['Adelie','Chinstrap'])]
penguins2C = penguins
penguins2C.head()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,species
0,39.1,18.7,181.0,3750.0,Adelie
1,39.5,17.4,186.0,3800.0,Adelie
2,40.3,18.0,195.0,3250.0,Adelie
4,36.7,19.3,193.0,3450.0,Adelie
5,39.3,20.6,190.0,3650.0,Adelie


In [25]:
y = penguins2C.pop('species').values
X_raw = penguins2C.values
feature_names = penguins2C.columns
X_tr_raw, X_ts_raw, y_train, y_test = train_test_split(X_raw, y, random_state=2, test_size=1/2)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_tr_raw)
X_test = scaler.transform(X_ts_raw)
max_k = X_train.shape[1]
X_train.shape, X_test.shape

((166, 4), (167, 4))

In [26]:
gnb = GaussianNB()
gnb.fit(X_train,y_train)
y_pred = gnb.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[68,  2,  0],
       [ 4, 30,  0],
       [ 0,  0, 63]], dtype=int64)

In [27]:
gnb = MyGaussianNB()
gnb.fit(X_train,y_train)
y_pred = gnb.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[68,  2,  0],
       [ 4, 30,  0],
       [ 0,  0, 63]], dtype=int64)